## Multinomial and Bernoulli Naive Bayes

For understanding Multinomial and Bernoulli Naive Bayes, we will start with a small example and understand the end to end process. In another notebook, we will build a full-fledged email spam classifier.

To start with, let's take a few sentences and classify them in two different classes - *education* or *cinema*. Each sentence will represent one document. In real-world cases, a document be any piece of text such as an email, a news article, a book review, a tweet etc.
The analysis and the algorithm involved doesn’t depend on the type of document we use.

The notebook is divided into the following sections:
1. Importing and preprocessing data
2. Building the model: Multinomial Naive Bayes
3. Building the model: Bernoulli Naive Bayes

### 1. Importing and Preprocessing Data

Let us first look at the sentences and their classes. We have kept the training sentences in file example_train.csv. Test sentences have been put in the file example_test.csv.

In [1]:
import pandas as pd

In [2]:
# Read Both the files ...

train_data = pd.read_csv('example_train.csv')
test_data = pd.read_csv('example_test.csv')

In [3]:
train_data

,Document,Class
0,Upgrad is a great educational institution.,education
1,Educational greatness depends on ethics,education
2,A story of great ethics and educational greatness,education
3,Sholey is a great cinema,cinema
4,good movie depends on good story,cinema


In [4]:
train_data['Class'] = train_data.Class.map({'cinema':0, 'education':1})

In [5]:
train_data.shape

(5, 2)

### Now lets convert this train_data to array because when we work with multinomial equations we have to pass the data in the array form.

In [6]:
train_array = train_data.values

In [7]:
train_array

array([['Upgrad is a great educational institution.', 1],
       ['Educational greatness depends on ethics', 1],
       ['A story of great ethics and educational greatness', 1],
       ['Sholey is a great cinema', 0],
       ['good movie depends on good story', 0]], dtype=object)

In [8]:
# Now we spliting data into X_train and y_train
X_train = train_array[:, 0]
y_train = train_array[:, 1]
y_train = y_train.astype('int')

### Creating the Bag of Words Representation

We now have to convert the data into a format which can be used for training the model. We'll use the **bag of words representation** for each sentence (document).

Imagine breaking X in individual words and putting them all in a bag. Then we pick all the unique words from the bag one by one and make a dictionary of unique words.

This is called **vectorization of words**. We have the class ```CountVectorizer()``` in scikit learn to vectorize the words.


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vec = CountVectorizer()

Here ```vec``` is an object of class ```CountVectorizer()```. This has a method called  ```fit()``` which converts a corpus of documents to a matrix of 'tokens'.

In [11]:
vec.fit(X_train)
vec.vocabulary_

{'upgrad': 15,
 'is': 9,
 'great': 6,
 'educational': 3,
 'institution': 8,
 'greatness': 7,
 'depends': 2,
 'on': 12,
 'ethics': 4,
 'story': 14,
 'of': 11,
 'and': 0,
 'sholey': 13,
 'cinema': 1,
 'good': 5,
 'movie': 10}

```Countvectorizer()``` has converted the documents into a set of unique words alphabetically sorted and indexed.


**Stop Words**

We can see a few trivial words such as  'and','is','of', etc. These words don't really make any difference in classyfying a document. These are called **stop words**. So we would like to get rid of them.

We can remove them by passing a parameter stop_words='english' while instantiating ```Countvectorizer()``` as follows:

In [12]:
# Now lets create another object and fit the model again.
vec = CountVectorizer(stop_words='english')
vec.fit(X_train)
vec.vocabulary_

{'upgrad': 11,
 'great': 5,
 'educational': 2,
 'institution': 7,
 'greatness': 6,
 'depends': 1,
 'ethics': 3,
 'story': 10,
 'sholey': 9,
 'cinema': 0,
 'good': 4,
 'movie': 8}

Notice that the vocabulary has reduced to 12 from 15. Another way of printing the 'vocabulary' is as follows:

Notice that the vocabulary has reduced to 12 from 15. Another way of printing the 'vocabulary' is as follows:

In [13]:
# printing feature names
print(vec.get_feature_names())
print(len(vec.get_feature_names()))

['cinema', 'depends', 'educational', 'ethics', 'good', 'great', 'greatness', 'institution', 'movie', 'sholey', 'story', 'upgrad']
12


In [14]:
# Lets print the features of the vectore
X_transformed = vec.transform(X_train)
X_transformed

<5x12 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

So our final dictionary is made of 12 words (after discarding the stop words). Now, to do classification, we need to represent all the documents with these words (or tokens) as features.

Every document will be converted into a *feature vector* representing presence of these words in that document. Let's convert each of our training documents in to a feature vector.

You can see X_tranformed is a 5 x 12 **sparse matrix**. It has 5 rows for each of our 5 documents and 12 columns each
for one word of the dictionary which we just created. Let us print X_transformed.

In [15]:
print(X_transformed)

  (0, 2)	1
  (0, 5)	1
  (0, 7)	1
  (0, 11)	1
  (1, 1)	1
  (1, 2)	1
  (1, 3)	1
  (1, 6)	1
  (2, 2)	1
  (2, 3)	1
  (2, 5)	1
  (2, 6)	1
  (2, 10)	1
  (3, 0)	1
  (3, 5)	1
  (3, 9)	1
  (4, 1)	1
  (4, 4)	2
  (4, 8)	1
  (4, 10)	1


This representation can be understood as follows:

Consider first 4 rows of the output: (0,2), (0,5), (0,7) and (0,11). It says that the first document (index 0) has
7th , 2nd , 5th and 11th 'word' present in the document, and that they appear only
once in the document- indicated by the right hand column entry.

Similarly, consider the entry (4,4) (third from bottom). It says that the fifth document has the fifth word present twice. Indeed, the 5th word('good') appears twice in the 5th document.

In real problems, you often work with large documents and vocabularies, and each document contains only a few words in the vocabulary. So it would be a waste of space to store the vocabulary in a typical dataframe, since most entries would be zero. Also, matrix products, additions etc. are much faster with sparse matrices. That's why we use sparse matrices to store the data.


Let us convert this sparse matrix into a more easily interpretable array:

Lets Print the Reguler venila representation of the X_transformed Array.

In [16]:
X_transformed.toarray()

array([[0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0]], dtype=int64)

To make the dataset more readable, let us examine the vocabulary and the document-term matrix together in a pandas dataframe. The way to convert a matrix into a dataframe is ```pd.DataFrame(matrix, columns=columns)```.


### Let's create a data using the X_transformed.

In [17]:
pd.DataFrame(X_transformed.toarray(), columns = vec.get_feature_names())

,cinema,depends,educational,ethics,good,great,greatness,institution,movie,sholey,story,upgrad
0,0,0,1,0,0,1,0,1,0,0,0,1
1,0,1,1,1,0,0,1,0,0,0,0,0
2,0,0,1,1,0,1,1,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,1,0,0
4,0,1,0,0,2,0,0,0,1,0,1,0


This table shows how many times a particular word occurs in document. In other words, this is a frequency table of the words.

A corpus of documents can thus be represented by a matrix with one row per document and one column per
token (e.g. word) occurring in the corpus.

Let's now import and transform the test data as well.

In [18]:
test_data.shape

(1, 2)

In [19]:
# Convert label to a numerical variable

test_data['Class'] = test_data.Class.map({'cinema':0, 'education':1})

In [20]:
test_data

,Document,Class
0,very good educational institution,1


In [21]:
# Lets create numpy array of the test data
test_array = test_data.values

In [22]:
# Lets Split the array data into X_test and y_test.
X_test = test_array[:, 0]
y_test = test_array[:, 1]
y_test = y_test.astype('int')

In [23]:
X_test

array(['very good educational institution'], dtype=object)

In [24]:
# transform the test data
# note that you *never* fit on test data, only on training data
# and only transform the test data
X_test_transformed = vec.transform(X_test)
X_test_transformed

<1x12 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [25]:
print(X_test_transformed)

  (0, 2)	1
  (0, 4)	1
  (0, 7)	1


In [26]:
X_test = X_test_transformed.toarray()

Let us summarise all we have done till now:

- ```vect.fit(train)``` learns the vocabulary of the training data
- ```vect.transform(train)``` uses the fitted vocabulary to build a document-term matrix from the training data
- ```vect.transform(test)``` uses the fitted vocabulary to build a document-term matrix from the testing data (and ignores tokens it hasn't seen before)

### 2. Building the Model: Multinomial Naive Bayes

In [27]:
# Import the model class from the sklearn module

from sklearn.naive_bayes import MultinomialNB

# Instantiating bernoulli NB class

mnb = MultinomialNB()

# Lets fit the model oon the X_transformed and y_train
mnb.fit(X_transformed, y_train)

# Let's predict the results...

proba = mnb.predict_proba(X_test)

In [28]:
# probability of each class (test data)
print("probability of test document belonging to class CINEMA" , proba[:,0])
print("probability of test document belonging to class EDUCATION" , proba[:,1])

probability of test document belonging to class CINEMA [0.32808399]
probability of test document belonging to class EDUCATION [0.67191601]


### 3. Building the Model: Bernoulli Naive Bayes

In [29]:
# Import the model class from the sklearn module

from sklearn.naive_bayes import BernoulliNB

# Instantiating bernoulli NB class

bnb = BernoulliNB()

# Lets fit the model oon the X_transformed and y_train
bnb.fit(X_transformed, y_train)

# Let's predict the results...

proba = bnb.predict_proba(X_test)

In [30]:
# probability of each class (test data)
print("probability of test document belonging to class CINEMA" , proba[:,0])
print("probability of test document belonging to class EDUCATION" , proba[:,1])

probability of test document belonging to class CINEMA [0.2326374]
probability of test document belonging to class EDUCATION [0.7673626]
